#### Introduction

This notebook creates maps and tables to determine the transit service at or near traffic signals. Currently, only signals on the State Highway Network (SHN) are shown.

#### How to use
On the calitp Jupyterhub, import `shared_utils` and authenticate to GCS.

#### Technical details (needs updating)
- Signal data is sourced from HQ Traffic Ops GIS, downloaded on 2025-09-08.
Segment data is calculated based on GTFS data collected by DDS for the selected targed date
- Routes are associated with a signal if they run within 155 meters of the signal. 
- Speeds and frequencies are defined based on data in the morning peak (7a-10a). Displayed speeds are the 20th percentile speeds.
- Network lines are based on data provided by Rebel. They are manually drawn, and do not strictly represent transit routes.
- Detailed speeds methodology is available at https://analysis.dds.dot.ca.gov/rt/README.html

#### Outputs
- Signal Map
  - Segments
    - 20th percentile speeds
    - Average frequencies
  - Signals
    - Unit Type
    - Aggregated data from 155 m to the signal
    - Route names (from `route_short_name`)
    - Organization names (from `organization_name`)
    - Total trips per hour, in both directions
    - IMMS ID
    - TMS ID
    - Delegation Type (describes whether a signal is owned/operated/maintaned by Caltrans or a local gov)
- Signal-Grain CSV
  - `signals_aggregated.csv`
    - Same data as in the interactive map
    - Additionally, the ArcGIS objectid is present to serve as a unique identifier
  - `signals_routes.csv`
    - One row for each route in each direction within 155 meters of each signal
    - Objectid
    - Route name
    - Organization name
    - Trips per hour in the chosen direction
    - TMS id
    - IMMS id
    - Delegation Type
    - Leased / Owned (whether the equipment is leased or owned by Caltrans)
    - Traffic Ops Comments
    
    
    

In [1]:
import branca
import geopandas as gpd
import pandas as pd
from calitp_data_analysis import gcs_geopandas, geography_utils
from shared_utils import catalog_utils, rt_utils, webmap_utils
from signal_tools import (
    add_transit_metrics_to_signals,
    filter_points_along_corridor,
    ready_signals_for_display,
    ready_speedmap_segments_for_display,
    sjoin_signals,
)
from constants import (
    TARGET_DATE,
    TARGET_TIME_OF_DAY,
    ANALYSIS_DISTRICT_NUMBER,
    TARGET_TIME_OF_DAY_LENGTH_HOURS,
    SJOIN_DISTANCE_METERS,
    SIGNAL_OWNER_NAME,
    STATE_OF_CALIFORNIA_LABEL,
    CITY_OF_LOS_ANGELES_LABEL,
    CITY_OF_SANTA_MONICA_LABEL,
    STUDY_CORRIDOR_BUFFER_DISTANCE,
    CALTRANS_INTERNAL_SIGNAL_URI,
    LOS_ANGELES_OPEN_SIGNAL_URI,
    SANTA_MONICA_OPEN_SIGNAL_URI,
)

In [2]:
g = gcs_geopandas.GCSGeoPandas()

### Get analysis corridors

In [3]:
study_corridors = gpd.read_file("study_corridors_lines.geojson")
study_corridors_albers = study_corridors.to_crs(geography_utils.CA_NAD83Albers_m)
study_corridors_buffered_wgs84 = study_corridors_albers.buffer(
    STUDY_CORRIDOR_BUFFER_DISTANCE
).to_crs(geography_utils.WGS84)

### Get Signal Data

In [4]:
# Get signal data from Caltrans Traffic Ops Data
caltrans_signals = (
    g.read_file(
        CALTRANS_INTERNAL_SIGNAL_URI,
    )
    .rename(columns=lambda s: s.lower())
    .clip(study_corridors_buffered_wgs84)
    # .clip(analysis_district)
)  # we want columns to be all lower case
# Filter out devices that aren't actually caltrans_signals
caltrans_signals_filtered = caltrans_signals.loc[
    caltrans_signals["tms_unit_type"] == "Traffic Signals"
].set_index("objectid")
caltrans_signals_filtered[SIGNAL_OWNER_NAME] = STATE_OF_CALIFORNIA_LABEL

In [5]:
# Get signal data from LADOT Open Data
ladot_signals = (
    g.read_file(LOS_ANGELES_OPEN_SIGNAL_URI)
    .to_crs(geography_utils.WGS84)
    .clip(study_corridors_buffered_wgs84)
)
ladot_signals_filtered = ladot_signals.loc[
    ladot_signals["ST3"] != "(SMART XWALK)"
].copy()
ladot_signals_filtered[SIGNAL_OWNER_NAME] = CITY_OF_LOS_ANGELES_LABEL

# Filter LADOT signals to avoid double counting signals with Caltrans signals
caltrans_signals_buffered = caltrans_signals_filtered.set_geometry(
    caltrans_signals_filtered.to_crs(geography_utils.CA_NAD83Albers_m)
    .buffer(
        100
    )  # TODO: Not sure if there's a good value for this buffer, might need to do manual work here
    .to_crs(geography_utils.WGS84)
)
ladot_signals_distinct_from_caltrans = ladot_signals_filtered.overlay(
    caltrans_signals_buffered, how="difference"
)

In [6]:
# Get signal data from Santa Monica Open Data
santa_monica_signals = g.read_file(SANTA_MONICA_OPEN_SIGNAL_URI).clip(
    study_corridors_buffered_wgs84
)
santa_monica_signals_albers = santa_monica_signals.to_crs(
    geography_utils.CA_NAD83Albers_m
).reset_index(drop=True)
santa_monica_signals_filtered = santa_monica_signals_albers.loc[santa_monica_signals_albers["circuit_nu"] != -9]
santa_monica_signals_clustered_by_circuit = santa_monica_signals_filtered.dissolve(
    "circuit_nu"
)
santa_monica_signals_duplicates_removed = (
    santa_monica_signals_clustered_by_circuit[
        [santa_monica_signals_filtered.geometry.name]
    ]
    .set_geometry(santa_monica_signals_clustered_by_circuit.geometry.centroid)
    .reset_index()
)
santa_monica_signals_duplicates_removed[SIGNAL_OWNER_NAME] = CITY_OF_SANTA_MONICA_LABEL

### Get Speedmaps

In [7]:
# read geo files
shared_data_catalog = catalog_utils.get_catalog("shared_data_catalog")
gtfs_data_constants = catalog_utils.get_catalog("gtfs_analytics_data")

# Get district polygons to mask
districts = shared_data_catalog.caltrans_districts.read()
analysis_district = districts.loc[districts["DISTRICT"] == ANALYSIS_DISTRICT_NUMBER]

In [8]:
# Get speedmap data
speedmap_segments = g.read_parquet(
    f"{gtfs_data_constants.speedmap_segments.dir}{gtfs_data_constants.speedmap_segments.segment_timeofday}_{TARGET_DATE}.parquet",
    filters=[
        ("time_of_day", "=", TARGET_TIME_OF_DAY)
    ],  # Filter for only a selected time of day
).clip(analysis_district)

In [9]:
speedmap_segments.head()

,schedule_gtfs_dataset_key,shape_id,shape_array_key,route_id,direction_id,stop_pair,segment_id,stop_pair_name,time_of_day,p50_mph,...,p80_mph,n_trips_sch,trips_hr_sch,route_short_name,geometry,name,base64_url,caltrans_district,analysis_name,source_record_id
526690,d27fcd9eec0803cce4dfdd2105aa5e8c,5004,608559ba37623a3456b9201e466e31cd,50,0.0,4097__4573,4097-4573-2,STUDEBAKER-WILLOW__KATELLA-CIVIC CENTER,AM Peak,39.95,...,40.53,8,2.667,50,"LINESTRING (-118.08902 33.80416, -118.08871 33...",OCTA Schedule,aHR0cHM6Ly9vY3RhLm5ldC9jdXJyZW50L2dvb2dsZV90cm...,12 - Santa Ana,Orange County Transportation Authority,reckk3FzZojulIKYL
341663,8e5034b0dddc8e5b0b0f04def813f271,930178,359dbea9995762ef154c52b0191388e5,93,1.0,1020__1021,1020-1021-1,7th & Park SE__7th & Federation SW,AM Peak,12.74,...,17.45,6,2.000,93,"LINESTRING (-118.13687 33.77535, -118.13683 33...",Long Beach Schedule,aHR0cHM6Ly9kcml2ZS5nb29nbGUuY29tL3VjP2V4cG9ydD...,07 - Los Angeles / Ventura,Long Beach Transit,recCv3CF4elAx0dUg
344351,8e5034b0dddc8e5b0b0f04def813f271,910200,b934ac07a0c0d3300000600b0c6c23bf,91,1.0,1020__1021,1020-1021-1,7th & Park SE__7th & Federation SW,AM Peak,13.61,...,16.05,6,2.000,91,"LINESTRING (-118.13687 33.77535, -118.13683 33...",Long Beach Schedule,aHR0cHM6Ly9kcml2ZS5nb29nbGUuY29tL3VjP2V4cG9ydD...,07 - Los Angeles / Ventura,Long Beach Transit,recCv3CF4elAx0dUg
348766,8e5034b0dddc8e5b0b0f04def813f271,460085,5efbfa048d2e3c3888c901b2c689b156,46,1.0,0538__1024,0538-1024-1,PCH & Anaheim NW__7th & Bellflower SW,AM Peak,14.00,...,20.47,8,2.667,46,"LINESTRING (-118.13298 33.78285, -118.13293 33...",Long Beach Schedule,aHR0cHM6Ly9kcml2ZS5nb29nbGUuY29tL3VjP2V4cG9ydD...,07 - Los Angeles / Ventura,Long Beach Transit,recCv3CF4elAx0dUg
345296,8e5034b0dddc8e5b0b0f04def813f271,410024,9495ea8824902c71ab5f47ac1162b701,41,1.0,0538__1024,0538-1024-1,PCH & Anaheim NW__7th & Bellflower SW,AM Peak,13.69,...,21.86,8,2.667,41,"LINESTRING (-118.13298 33.78285, -118.13293 33...",Long Beach Schedule,aHR0cHM6Ly9kcml2ZS5nb29nbGUuY29tL3VjP2V4cG9ydD...,07 - Los Angeles / Ventura,Long Beach Transit,recCv3CF4elAx0dUg


### Merge signal and speedmap info

In [10]:
caltrans_sjoined_signals_segments = sjoin_signals(
    signals_gdf=caltrans_signals_filtered.reset_index(),
    segments_lines_gdf=speedmap_segments,
    # keep_columns_signals=["objectid", "imms_id"],
    # keep_columns_segments=["shape_id", "segment_id", "organization_name", "route_id"],
    unique_identifier_signals="objectid",
    signals_buffer_distance=SJOIN_DISTANCE_METERS,
)
caltrans_signals_with_transit_metrics = add_transit_metrics_to_signals(
    signals_gdf=caltrans_signals_filtered.reset_index(),
    sjoined_signals_segments=caltrans_sjoined_signals_segments,
    unique_identifier_signals="objectid",
)

ladot_sjoined_signals_segments = sjoin_signals(
    signals_gdf=ladot_signals_distinct_from_caltrans,
    segments_lines_gdf=speedmap_segments,
    unique_identifier_signals="ID",
    signals_buffer_distance=SJOIN_DISTANCE_METERS,
)
ladot_signals_with_transit_metrics = add_transit_metrics_to_signals(
    signals_gdf=ladot_signals_distinct_from_caltrans,
    sjoined_signals_segments=ladot_sjoined_signals_segments,
    unique_identifier_signals="ID",
)

santa_monica_sjoined_signals_segments = sjoin_signals(
    signals_gdf=santa_monica_signals_duplicates_removed,
    segments_lines_gdf=speedmap_segments,
    unique_identifier_signals="circuit_nu",
    signals_buffer_distance=SJOIN_DISTANCE_METERS,
)
santa_monica_signals_with_transit_metrics = add_transit_metrics_to_signals(
    signals_gdf=santa_monica_signals_duplicates_removed,
    sjoined_signals_segments=santa_monica_sjoined_signals_segments,
    unique_identifier_signals="circuit_nu",
)

### Concatenate segments

In [11]:
# Define columns to include
arrowized_segments_display_columns = [
    "trips_hr_sch",
    "p50_mph",
    "p20_mph",
    "p80_mph",
    "route_short_name",
    "stop_pair_name",
    "segment_id",
    "route_id",
    "shape_id",
]
speedmaps_signals_together = gpd.GeoDataFrame(
    pd.concat(
        [
            caltrans_sjoined_signals_segments,
            ladot_sjoined_signals_segments,
            santa_monica_sjoined_signals_segments,
        ],
    ).drop_duplicates(subset=arrowized_segments_display_columns)
)

### Get GDFs formatted for display on the webmap

In [12]:
transit_info_columns = [
    "route_names_aggregated",
    "analysis_names_aggregated",
    "Trips/Hour",
    "signal_owner",
    "color_key"
]
caltrans_signals_with_transit_display_columns = [
    "tms_unit_type",
    *transit_info_columns,
    # "asset_sub_type",
    "tms_id",
    "imms_id",
    "delegation_type",
    # "leased_owned",
    # "comment",
    caltrans_signals_with_transit_metrics.geometry.name,
]
ladot_signals_with_transit_display_columns = [
    "ID",
    *transit_info_columns,
    ladot_signals_with_transit_metrics.geometry.name,
]
santa_monica_signals_with_transit_display_columns = [
    "circuit_nu",
    *transit_info_columns,
    santa_monica_signals_with_transit_metrics.geometry.name,
]

In [13]:
# Segment GDF
arrowized_gdf = ready_speedmap_segments_for_display(speedmaps_signals_together)
# Signal GDF
ownership_map = {
    STATE_OF_CALIFORNIA_LABEL: 0,
    CITY_OF_LOS_ANGELES_LABEL: 1,
    CITY_OF_SANTA_MONICA_LABEL: 2
}
signal_size = 50
caltrans_buffered_signals = ready_signals_for_display(
    caltrans_signals_with_transit_metrics, buffer_distance=signal_size, ownership_map=ownership_map
)
ladot_buffered_signals = ready_signals_for_display(
    ladot_signals_with_transit_metrics, buffer_distance=signal_size, ownership_map=ownership_map
)
santa_monica_buffered_signals = ready_signals_for_display(
    santa_monica_signals_with_transit_metrics, buffer_distance=signal_size, ownership_map=ownership_map
)

In [14]:
# Get study corridors to add onto the map
study_corridors_for_display = study_corridors_albers.set_geometry(
    study_corridors_albers.buffer(100, cap_style="flat")
)
study_corridors_for_display["bus_lane"] = study_corridors["bus_lane"].fillna(0)
# Get colormap based on whether the analysis segment is a bus lane
DDS_GREY = "#d9d9d6"
DDS_BLUE = "#b1e4e3"
study_corridor_colormap = branca.colormap.StepColormap(
    colors=[DDS_GREY, DDS_BLUE], index=[0, 1]
)

In [15]:
# Signals colormap
DDS_MIDDLE_PURPLE = "#7474c1"
DDS_MIDDLE_ORANGE = "#ff8200"
DDS_MIDDLE_BLUE = "#00a3e0"
signal_colorscale = branca.colormap.StepColormap(
    colors=[DDS_MIDDLE_PURPLE, DDS_MIDDLE_ORANGE, DDS_MIDDLE_BLUE],
    vmin=0, vmax=2
)

In [16]:
signal_colorscale(2)

'#00a3e0ff'

### Create webmap

In [17]:
SIGNAL_LEGEND_URL = "https://storage.googleapis.com/calitp-map-tiles/westside_signals_legend.svg"
signal_folder = "signals_v12_52/"

# Study corridors
study_corridor_map = webmap_utils.set_state_export(
    study_corridors_for_display,
    subfolder=signal_folder,
    filename="study_corridors",
    cmap=study_corridor_colormap,
    color_col="bus_lane",
    map_title="Study Corridors",
)

# Speeds
speedmap = webmap_utils.set_state_export(
    arrowized_gdf[[*arrowized_segments_display_columns, arrowized_gdf.geometry.name]],
    subfolder=signal_folder,
    filename="speeds",
    cmap=rt_utils.ACCESS_ZERO_THIRTY_COLORSCALE,
    color_col="p20_mph",
    cache_seconds=1,
    map_type="new_speedmap",
    # legend_url=rt_utils.SPEEDMAP_LEGEND_URL,
    map_title="Speeds",
    existing_state=study_corridor_map["state_dict"],
)

# Signals
# Caltrans
caltrans_signal_speedmap = webmap_utils.set_state_export(
    caltrans_buffered_signals[caltrans_signals_with_transit_display_columns],
    subfolder=signal_folder,
    cmap=signal_colorscale,
    color_col="color_key",
    existing_state=speedmap["state_dict"],
    map_title="ct_signals",
    filename="ct_signals"
    # legend_url=SIGNAL_LEGEND_URL,
    # manual_centroid=[34.048108, -118.4183252],
)
ladot_signal_speedmap = webmap_utils.set_state_export(
    ladot_buffered_signals[ladot_signals_with_transit_display_columns],
    subfolder=signal_folder,
    cmap=signal_colorscale,
    color_col="color_key",
    existing_state=caltrans_signal_speedmap["state_dict"],
    map_title="ladot_signals",
    filename="ladot_signals"
    # legend_url=SIGNAL_LEGEND_URL,
    # manual_centroid=[34.048108, -118.4183252],
)
santa_monica_signal_speedmap = webmap_utils.set_state_export(
    santa_monica_buffered_signals[santa_monica_signals_with_transit_display_columns],
    subfolder=signal_folder,
    cmap=signal_colorscale,
    color_col="color_key",
    existing_state=ladot_signal_speedmap["state_dict"],
    map_title=f"Signals with Approach Speeds {TARGET_DATE}",
    filename="santa_monica_signals",
    legend_url=SIGNAL_LEGEND_URL,
    manual_centroid=[34.048108, -118.4183252],
)
santa_monica_signal_speedmap

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:124: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())
/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:124: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


file already exists at calitp-map-tiles/signals_v12_52/study_corridors.geojson.gz, not overwriting, use overwrite = True to overwrite
file already exists at calitp-map-tiles/signals_v12_52/speeds.geojson.gz, not overwriting, use overwrite = True to overwrite


/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:124: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())
/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:124: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


file already exists at calitp-map-tiles/signals_v12_52/ct_signals.geojson.gz, not overwriting, use overwrite = True to overwrite
file already exists at calitp-map-tiles/signals_v12_52/ladot_signals.geojson.gz, not overwriting, use overwrite = True to overwrite
file already exists at calitp-map-tiles/signals_v12_52/santa_monica_signals.geojson.gz, not overwriting, use overwrite = True to overwrite


{'state_dict': {'name': 'null',
  'layers': [{'name': 'Study Corridors',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/signals_v12_52/study_corridors.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}},
   {'name': 'Speeds',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/signals_v12_52/speeds.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'new_speedmap'},
   {'name': 'ct_signals',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/signals_v12_52/ct_signals.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}},
   {'name': 'ladot_signals',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/signals_v12_52/ladot_signals.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}},
   {'name': 'Signals with Approach Speeds 2025-09-24',
    'url'

In [18]:
ladot_buffered_signals[SIGNAL_OWNER_NAME]

0      City of Los Angeles
1      City of Los Angeles
2      City of Los Angeles
3      City of Los Angeles
4      City of Los Angeles
              ...         
189    City of Los Angeles
190    City of Los Angeles
191    City of Los Angeles
192    City of Los Angeles
193    City of Los Angeles
Name: signal_owner, Length: 194, dtype: object

### Get tabular data (ct signals only)

In [19]:
# Get signal-route grain data
signal_route_group = caltrans_sjoined_signals_segments.groupby(
    ["objectid", "route_short_name", "organization_name", "direction_id"]
)

signals_routes_frequency = signal_route_group["trips_hr_sch"].sum()
merged_signals_routes_frequency = signals_routes_frequency.reset_index().merge(
    caltrans_signals_filtered[
        [
            "tms_unit_type",
            "asset_sub_type",
            "location",
            "tms_id",
            "imms_id",
            "delegation_type",
            "leased_owned",
            "comment",
            "geometry",
        ]
    ],
    how="left",
    left_on="objectid",
    right_index=True,
    validate="many_to_one",
)
merged_geometry = gpd.GeoSeries(merged_signals_routes_frequency["geometry"]).to_crs(
    geography_utils.WGS84
)
merged_signals_routes_frequency["latitude"] = merged_geometry.y.round(5)
merged_signals_routes_frequency["longitude"] = merged_geometry.x.round(5)
merged_signals_routes_frequency.drop("geometry", axis=1).to_csv(
    "signals_routes.csv", index=False
)

KeyError: 'organization_name'

In [ ]:
# Save signal-grain data to a csv
caltrans_signals_with_transit_metrics["latitude"] = caltrans_signals_with_transit_metrics.geometry.y.round(5)
caltrans_signals_with_transit_metrics["longitude"] = caltrans_signals_with_transit_metrics.geometry.x.round(5)
caltrans_signals_with_transit_metrics.rename(columns={"trips_hr_sch": "Trips/Hour"})[
    [
        "tms_unit_type",
        "route_names_aggregated",
        "organization_names_aggregated",
        "Trips/Hour",
        "latitude", 
        "longitude",
        "comment",
        "tms_id",
        "imms_id",
        "delegation_type",
    ]
].to_csv("signals_aggregated.csv")